In [12]:
import pandas as pd
import numpy as np
import keras_nlp
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

## 1. Loading the dataset

In [13]:
df = pd.read_csv('../../data/processed/001_processed_df.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   summary  2045 non-null   object
 1   label    2045 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 32.1+ KB


In [15]:
df.head()

,summary,label
0,Barracuda Brigade muslims busted stole million...,1
1,reasoning with facts attorney general loretta ...,1
2,Barracuda Brigade breaking weiner cooperating ...,1
3,Fed Up pin drop speech father daughter kidnapp...,1
4,Fed Up fantastic trumps point plan reform heal...,1


## 2. Loading BERT Model base

## 3. Building the Model

### 3.1. Spliting the dataset for the trainning

In [16]:
x=df['summary']
y=df['label']

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42, stratify=y)

### 3.2. Scaffolding the model layers

In [17]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased",trainable=True)
  encoder_inputs = preprocessor(text_input)
  encoder = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [18]:
text_test = ['this is such an amazing movie!']

classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.35506427]], shape=(1, 1), dtype=float32)


In [19]:
classifier_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)   │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_preprocessor_1 │ [(None, 512),     │          0 │ text[0][0]        │
│ (BertPreprocessor)  │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_backbone       │ [(None, 768),     │ 109,482,2… │ bert_preprocesso… │
│ (BertBackbone)      │ (None, 512, 768)] │            │ bert_preprocesso… │
│                     │                   │            │ bert_preprocesso… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_25          │ (None, 768)       │          0 │ bert_backbone[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classifier (Dense)  │ (None, 1)         │        769 │ dropout_25[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 109,483,009 (417.64 MB)

 Trainable params: 109,483,009 (417.64 MB)

 Non-trainable params: 0 (0.00 B)

### 3.3. Adding lost functions and metrics

In [20]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

In [21]:
classifier_model.compile(optimizer='adam',
                         loss='binary_crossentropy',
                         metrics=METRICS)

### 3.4. Fitting the model

In [22]:
history = classifier_model.fit(X_train, y_train, epochs=10)

Epoch 1/10


52/52 ━━━━━━━━━━━━━━━━━━━━ 7915s 151s/step - accuracy: 0.3830 - loss: 9.7692 - precision: 0.3565 - recall: 0.9072
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 8152s 157s/step - accuracy: 0.3858 - loss: 9.7913 - precision: 0.3858 - recall: 1.0000
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 12220s 236s/step - accuracy: 0.3858 - loss: 9.7912 - precision: 0.3858 - recall: 1.0000
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 13462s 261s/step - accuracy: 0.3604 - loss: 10.1975 - precision: 0.3604 - recall: 1.0000
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 10192s 197s/step - accuracy: 0.3525 - loss: 10.3221 - precision: 0.3525 - recall: 1.0000
Epoch 6/10
16/52 ━━━━━━━━━━━━━━━━━━━━ 3:46:35 378s/step - accuracy: 0.4000 - loss: 9.5650 - precision: 0.4000 - recall: 1.0000